**Downloading Required Libraries**

In [1]:
!pip install rake-nltk
!pip install transformers
!pip install gensim==3.4.0 

**Importing Required Libraries**

In [2]:
import numpy as np 
import pandas as pd 
import os
import numpy as np
import torch
from tqdm import tqdm
from rake_nltk import Rake
import nltk
import re
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
from transformers import GPT2Tokenizer, GPT2Model
from gensim.summarization.summarizer import summarize
from ignite.metrics import Rouge
from ignite.metrics.nlp import Bleu

New version of Gensim don't have summerize module, so using old version. 

**Downloading Stopward and Punctuations for english vocabulary**

In [5]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Loading and preprocessing Data**

In [7]:
#DIR = "/content/drive/MyDrive/Story_Generation_ANLP/"
DIR="/story-generation/Story_Generation_ANLP"
data = [DIR+"valid", DIR+"test" , DIR+"train" ]
target_data = [DIR+"valid", DIR+"test", DIR+"train"]

**Converting Dataset in appropriate Form:**
Creating Dataset in appropriate format as mentioned in the paper. Every data instance have an input "Prompt" and corresponding story line. Which at this phase is converted to [ WP ] PROMPT <endprompt> OUTLINE <endoutline> TOP 10 KEYWORD/ PHRASE. This is one time step and run at the first iteration. So commenting the code out.

In [8]:
# for path in data:
#   fp = open(path + ".wp_source") 
#   ft = open(path + ".wp_target")

#   stories = ft.readlines()  
#   prompts = fp.readlines()

#   #rstrip: Remove any white spaces at the end of the string
#   #we need to get outline from the prompt
#   i=0
#   new_stories=[]

#   for i in tqdm(range(len(stories))):
    
#     story=stories[i]
#     prompt=prompts[i]

#     r = Rake()
#     r.extract_keywords_from_text(story)

#     keywordStory=""
#     count=0

#     for word in r.get_ranked_phrases():
#       """ Ranked phrases return a list of phrases, each phrases can have mutliple words"""
#       if count==10: # According to (Yao et al., 2019) and the average lengths of stories in our corpus, we extract 10 keywords for each story.
#         break
#       count+=1
#       keywordStory+=word
    
#     storyRemovingNewline=re.sub("<newline>","\n",story)

#     try:
#       summary=summarize(storyRemovingNewline,ratio=0.3)
#       summary=re.sub("\n"," ",summary)
#       new_stories.append(prompt.rstrip() + " <endprompts> " + summary.rstrip() +" <endoutline> "+keywordStory)
#     except:
#       pass
    
#   with open(path + ".wp_combined", "w") as o:
#     for line in new_stories:
#       o.write(line.strip() + "\n")
#     print('finish writing',path + ".wp_combined")

#   fp.close()
#   ft.close()

When running the session first time, copy the dataset to working directory, as input directory is read only, since kaggle input directory is read only

In [9]:
#!rm -rf /kaggle/working/*
!cp -r /kaggle/input/ /kaggle/working

In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"

Creating Dataset in appropriate Fromat using Tokenizer and Defining train function for GPT2 finetuning.

In [11]:
def load_dataset(file_path, tokenizer, block_size = 128):
    return TextDataset(tokenizer = tokenizer,file_path = file_path,block_size = block_size,)

def load_data_collator(tokenizer, mlm = False):
    return DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=mlm,)
    
def train(train_file_path,val_file_path,model_name,output_dir,per_device_train_batch_size,per_device_eval_batch_size,num_train_epochs,save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  data_collator = load_data_collator(tokenizer)
  train_dataset = load_dataset(train_file_path, tokenizer)
  val_dataset = load_dataset(val_file_path,tokenizer)
    
  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name) 

  """
    using cross entropy to train on
    https://github.com/huggingface/transformers/blob/391db836ab7ed2ca61c51a7cf1b135b6ab92be58/transformers/modeling_gpt2.py#L539
  """

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(output_dir=output_dir,
                                    per_device_train_batch_size=per_device_train_batch_size,
                                    per_device_eval_batch_size=per_device_eval_batch_size,
                                    num_train_epochs=num_train_epochs,
                                    evaluation_strategy="steps",eval_steps=10000,save_strategy="no")
    
  trainer = Trainer(model=model,args=training_args,data_collator=data_collator,
                    train_dataset=train_dataset,eval_dataset=val_dataset)
      
  trainer.train()
  trainer.save_model()

**Hyperparameters**

In [12]:
# you need to set parameters 
path="/kaggle/working/input/story-generation/Story_Generation_ANLP/"
train_file_path = path+'/train.wp_combined'
validation_file_path= path+'/valid.wp_combined'
model_name = 'gpt2'
output_dir = '/kaggle/working/'
per_device_train_batch_size = 16
num_train_epochs = 1 
save_steps = 100000

**Training**

In [14]:
train(
    train_file_path=train_file_path,
    val_file_path=validation_file_path,
    model_name=model_name,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 732746
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 45797


Step,Training Loss,Validation Loss
10000,3.716700,3.637604
20000,3.668300,3.597819
30000,3.648300,3.576489
40000,3.618600,3.563454


***** Running Evaluation *****
  Num examples = 41875
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41875
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41875
  Batch size = 16
***** Running Evaluation *****
  Num examples = 41875
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /kaggle/working/
Configuration saved in /kaggle/working/config.json
Model weights saved in /kaggle/working/pytorch_model.bin


**Evaluating Performance of the model**

In [22]:
model_path = '/kaggle/working/'
FinalModel = GPT2LMHeadModel.from_pretrained(model_path)
FinalTokenizer = GPT2Tokenizer.from_pretrained(model_path)

loading configuration file /kaggle/working/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading weigh

In [31]:
def inference(sentence,maxlen=500):

    ids = FinalTokenizer.encode(f'{sentence}', return_tensors='pt')

    final_outputs = FinalModel.generate(ids,do_sample=True,max_length=maxlen,
        pad_token_id=FinalModel.config.eos_token_id,top_k=50,top_p=0.95,)

    return FinalTokenizer.decode(final_outputs[0], skip_special_tokens=True)

**Inference**

In [77]:
testPathS="/kaggle/working/input/story-generation/Story_Generation_ANLP/test.wp_source"
testPathT="/kaggle/working/input/story-generation/Story_Generation_ANLP/test.wp_target"

blue = Bleu(ngram=4, smooth="smooth1")
rougel = Rouge(variants=["L", 2], multiref="best")
final={'Rouge-L-P': 0.0, 'Rouge-L-R': 0.0, 'Rouge-L-F': 0.0, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0}
finalBlue=0
counter=0

def evaluateSentence():
    fileS = open(testPathS)
    fileT = open(testPathT)
    sourceLine=fileS.readlines()
    targetLine=fileT.readlines()
    finalBlue=0
    length=0
    for counter in tqdm(range(100)): 
        sequence=inference(sourceLine[counter],500)
        rougel.update(([sequence.split()], [targetLine[counter].split()]))
        a=rougel.compute()
        y=[targetLine[counter]]
        blue.update(([sequence.split()],[[_y.split() for _y in y]]))
        finalBlue=finalBlue+blue.compute()
        
        for keys, value in a.items():
            final[keys]+=a[keys]
        
        length+=1
    
    fileS.close()
    fileT.close()
    return finalBlue,length

In [78]:
finalBlue,length=evaluateSentence()
print(final)#estimated time for all test samples
print("Blue score: ",finalBlue)
print(length)

100%|██████████| 100/100 [49:12<00:00, 29.52s/it]

{'Rouge-L-P': 0.2799722682867467, 'Rouge-L-R': 100.0, 'Rouge-L-F': 100.0, 'Rouge-2-P': 0.0011308167830883036, 'Rouge-2-R': 0.22955580696692568, 'Rouge-2-F': 0.22955580696692568}
Blue score:  tensor(0.4290, dtype=torch.float64)
100


**Interactive Console**

In [82]:
sequence = input() 
print(inference(sequence))

 Titanic My Heart Will Go on


Titanic My Heart Will Go on, I 'll wake up on my bed. My only friends are these men, those are the things I could never truly trust. They would all be gone in a matter of weeks, they just don't seem like the right type. <endoutline> time someone else findsgood guys always lookstray cat came upontitanic my heart will go`` hey guys back`` welcome back`` hello... ''old enough nowadays`` hey buddy`` hey buddy
[ WP ] `` You don't even seem to know me anymore. I'm from my past and it doesn't matter anymore, so why waste my time with a stranger? '' <endprompts> '' he said, his eyes darted in and out of my view and I could tell he was staring right at me. I was trying to keep a secret of some sort, I think but there was something else I wished he would tell me, he was in my presence, but I didn't have time to take that. I knew he was holding me at his hand and was very tall and strong and strong like a great big boy, but I felt like I was stronger than that, not as far out there was a boy but 